In [1]:

import numpy as np
import data_functions
import gensim

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster.hierarchical import AgglomerativeClustering

from numpy.linalg import norm
from gensim import corpora

import nltk
import pickle
n_clusters=15


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
questions_answers, questions_original, answers_original, questions_norm=data_functions.load_data()


loading data, please wait...
loading data DONE


In [3]:
#сколько слов в словаре
def create_dictionary(questions_normalized, low_limit, max_limit):
    """
    Готовим словарь из нормированных вопров    
    """
    words_all_norm=[]
    for question in questions_normalized:
        for word in question:
            words_all_norm.append(word)
    print ('Всего нормализованных слов:', len((words_all_norm)))
    words_uniq_norm=set(words_all_norm)
    print ('Всего уникальных нормализованных слов:',len((words_uniq_norm)))
    
    words_all_norm_freq = nltk.probability.FreqDist(words_all_norm)
    dictionary_words=[]
    for word in words_all_norm_freq:
        if words_all_norm_freq[word]>low_limit and words_all_norm_freq[word]<max_limit:
            dictionary_words.append(word)
    print ('Всего слов в словаре:',len(dictionary_words))
    print ('Отброшены слова повторяемостью меньше {} и больше {}'.format(low_limit, max_limit))
    return dictionary_words

dictionary=create_dictionary(questions_norm,10,3000)

Всего нормализованных слов: 633179
Всего уникальных нормализованных слов: 18488
Всего слов в словаре: 3307
Отброшены слова повторяемостью меньше 10 и больше 3000


In [4]:
#отобразить перечень слов упорядоченных по повторяемости
def get_top_freq_words(words_all_norm, top_positions):
    words_all_norm_freq = nltk.probability.FreqDist(words_all_norm)
    for w in sorted(words_all_norm_freq, key=words_all_norm_freq.get, reverse=True)[:top_positions]:
        print (w, words_all_norm_freq[w])
    return

In [5]:
get_top_freq_words(words_all_norm, 10)

NameError: name 'words_all_norm' is not defined

In [ ]:
dictionary = corpora.Dictionary([dictionary_words])
corpus = [dictionary.doc2bow(text) for text in [x for x in questions_norm]]
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
print ('Calculating model...')
ldamodel = gensim.models.ldamodel.LdaModel.load('model.gensim')
#ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = n_clusters, id2word=dictionary, passes=30)
print ('Calculating model... DONE')
ldamodel.save('model.gensim')

topics = ldamodel.print_topics(num_words=10)

In [ ]:
#вывести топики
for topic in topics:
    print(topic)
for t, top_words in ldamodel.print_topics(num_topics=10, num_words=10):
    print ("Topic", t, ":", top_words)

In [ ]:
#определение ближайших 
current_question='ком звон на представл сотрудник служб социаологическ пользован услуг сбербанк то мне сегодн одн девушк позвон номер 7499xxxxxxx ли удел минут на эт ну соглас на что он зада мне вопрос насчет тог есл за последн месяц мо организац пользова ег услуг ил тог ответ что нет посл чег кто эт мог быт действительн ли провод так опрос кто'

current_question = data_functions.clean_text(current_question)
current_question_bow = dictionary.doc2bow(current_question)
#print(current_question_bow)
print(ldamodel.get_document_topics(current_question_bow))

In [ ]:
#проставляем всем вопросам темы
#todo - скорее всего - вычисление еще раз doc2bow для каждого нормированного вопроса - не очень удачная идея
question_topics_idx=[]
for question in questions_norm[:]:
    themes_and_probs_in_question=ldamodel.get_document_topics(dictionary.doc2bow(question))
    themes_probs_in_question=[i[1] for i in themes_and_probs_in_question] 
    question_topics_idx.append(themes_and_probs_in_question[np.argmax(themes_probs_in_question)][0])
len(question_topics_idx)

In [ ]:
#отразить количество вопрсов в кластерах
for theme_num in range(n_clusters):
    count=np.array(np.where(np.array(question_topics_idx) == theme_num)).size
    count_perc=100.*count/len(question_topics_idx)
    print ("Размер темы "+ str(theme_num) + ': ' + str(count) + ' %= ' + str(int(count_perc)))

In [ ]:
import pyLDAvis.gensim

In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model.gensim')

lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display, local=True)